<a href="https://colab.research.google.com/github/SergioMartinezCid/MUII-DSI-2022-23/blob/master/Clustering/DSI_Trabajo_Cl%C3%BAstering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO
* Hacer agregación de goles, fallos y efectividad
* EGGoals y EGMissed considerar quitarlos por falta de datos (wtf? tiene bastantes)
* Time, scoring, prod100 son complicados y probar con ellos y sin ellos

# Importación de Datos

In [1]:
!wget -O CSVORIGINAL.csv https://raw.githubusercontent.com/SergioMartinezCid/MUII-DSI-2022-23/master/Clustering/CSVORIGINAL.csv

--2023-02-21 14:34:48--  https://raw.githubusercontent.com/SergioMartinezCid/MUII-DSI-2022-23/master/Clustering/CSVORIGINAL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350830 (343K) [text/plain]
Saving to: ‘CSVORIGINAL.csv’

CSVORIGINAL.csv     100%[===================>] 342.61K  --.-KB/s    in 0.04s   

2023-02-21 14:34:48 (9.11 MB/s) - ‘CSVORIGINAL.csv’ saved [350830/350830]



In [2]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
original_df = pd.read_csv('CSVORIGINAL.csv', delimiter='\t', decimal=',')

In [5]:
# Se borra la última fila, que es nula
original_df = original_df.drop(3071, axis=0)
# Se elimina la columna prod100 por tener información agregada con scoring
original_df = original_df.drop(['prod100'], axis=1)
# Se eliminan las columnas de las tarjetas roja, amarilla y azul por no tener información relevante (hay muy pocas situaciones en las que se haya sacado tarjeta)
original_df = original_df.drop(['YC', 'RC', 'BC'], axis=1)
# Comprobamos si ha quedado algún valor nulo
original_df.isnull().values.any()
original_df.dtypes

KeyError: ignored

# Identificar los porteros

In [ ]:
#Agrupamos por jugador
# Quitamos Phase, Match No., Match, No. y Team por ser columnas identificativas que no son relevantes para los jugadores
player_df = original_df.drop(['Phase', 'Match No.', 'Match', 'No.', 'Team'], axis=1)
# Establecer la columna Name como index porque es una columna identificativa
player_df = player_df.groupby(['Name'], as_index=False).sum()
#Seleccionamos los porteros que serán las filas que tienen tiempo pero 0 scoring
only_goalkeepers = player_df[(player_df.scoring==0)&(player_df.Time > 0)]
only_goalkeepers.head()

,Name,7mGoals,7mMissed,6mGoals,6mMissed,WingGoals,WingMissed,BTGoals,BTMissed,FBGoals,...,EGMissed,9mGoals,9mMissed,TF,2Min,AS,ST,BS,Time,scoring
7,ABIDI Mohamed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146.783333,0.0
16,ALDERI Ali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.866667,0.0
23,ALMEIDA Cesar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.533333,0.0
24,ALMEIDA Luis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.466667,0.0
28,ALSHUWAIKH Qasim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.733333,0.0


# Agregación de Datos por Equipo

## Incluyendo porteros

In [ ]:
# Quitamos Phase, Match No., Match, No. y Name por ser columnas identificativas que no son relevantes para los equipos
team_goalkeepers_df = original_df.drop(['Phase', 'Match No.', 'Match', 'No.', 'Name'], axis=1)
# Quitamos Time porque para los equipos no es relevante porque jugarán más los que lleguen a la final
team_goalkeepers_df = team_goalkeepers_df.drop('Time',axis=1)
# Establecer la columna Team como index porque es una columna identificativa
team_goalkeepers_df = team_goalkeepers_df.groupby(['Team'], as_index=True).sum()
team_goalkeepers_df.head()

,7mGoals,7mMissed,6mGoals,6mMissed,WingGoals,WingMissed,BTGoals,BTMissed,FBGoals,FBMissed,EGGoals,EGMissed,9mGoals,9mMissed,TF,2Min,AS,ST,BS,scoring
Team,,,,,,,,,,,,,,,,,,,,
ALG,16.0,6.0,31.0,18.0,16.0,16.0,28.0,8.0,9.0,7.0,6.0,2.0,34.0,72.0,63.0,26.0,50.0,21.0,3.0,38.80
ANG,16.0,14.0,38.0,11.0,20.0,9.0,25.0,4.0,11.0,7.0,2.0,1.0,51.0,70.0,72.0,25.0,68.0,13.0,9.0,80.80
ARG,10.0,5.0,34.0,17.0,15.0,17.0,16.0,7.0,22.0,5.0,7.0,2.0,16.0,30.0,63.0,20.0,48.0,9.0,3.0,41.80
AUT,24.0,6.0,52.0,12.0,31.0,14.0,38.0,6.0,33.0,11.0,5.0,3.0,37.0,55.0,81.0,33.0,100.0,34.0,12.0,176.65
BLR,9.0,2.0,45.0,20.0,33.0,17.0,19.0,4.0,17.0,3.0,4.0,2.0,44.0,53.0,58.0,30.0,65.0,12.0,13.0,117.30


## Excluyendo porteros

In [ ]:
# Eliminamos los porteros
team_no_goalkeepers_df = original_df.copy()
team_no_goalkeepers_df = team_no_goalkeepers_df[~team_no_goalkeepers_df['Name'].isin(only_goalkeepers['Name'])]
# Quitamos Phase, Match No., Match, No. y Name por ser columnas identificativas que no son relevantes para los equipos
team_goalkeepers_df = original_df.drop(['Phase', 'Match No.', 'Match', 'No.', 'Name'], axis=1)
# Quitamos Time porque para los equipos no es relevante porque jugarán más los que lleguen a la final
team_no_goalkeepers_df = team_no_goalkeepers_df.drop('Time',axis=1)
# Establecer la columna Team como index porque es una columna identificativa
team_no_goalkeepers_df = team_no_goalkeepers_df.groupby(['Team'], as_index=True).sum()
team_no_goalkeepers_df.head()

,Match No.,No.,7mGoals,7mMissed,6mGoals,6mMissed,WingGoals,WingMissed,BTGoals,BTMissed,...,EGGoals,EGMissed,9mGoals,9mMissed,TF,2Min,AS,ST,BS,scoring
Team,,,,,,,,,,,,,,,,,,,,,
ALG,3716.0,2287.0,16.0,6.0,31.0,18.0,16.0,16.0,28.0,8.0,...,6.0,2.0,34.0,72.0,63.0,26.0,50.0,21.0,3.0,38.80
ANG,5670.0,1388.0,16.0,14.0,38.0,11.0,20.0,9.0,25.0,4.0,...,2.0,1.0,51.0,70.0,72.0,25.0,68.0,13.0,9.0,80.80
ARG,3584.0,1797.0,10.0,5.0,34.0,17.0,15.0,17.0,16.0,7.0,...,7.0,2.0,16.0,30.0,63.0,20.0,48.0,9.0,3.0,41.80
AUT,6864.0,4127.0,24.0,6.0,52.0,12.0,31.0,14.0,38.0,6.0,...,5.0,3.0,37.0,55.0,81.0,33.0,100.0,34.0,12.0,176.65
BLR,3907.0,3867.0,9.0,2.0,45.0,20.0,33.0,17.0,19.0,4.0,...,4.0,2.0,44.0,53.0,58.0,30.0,65.0,12.0,13.0,117.30


# Agregación de Datos por Jugador

In [ ]:
# Quitamos Phase, Match No., Match, No. y Team por ser columnas identificativas que no son relevantes para los jugadores
player_df = original_df.drop(['Phase', 'Match No.', 'Match', 'No.', 'Team'], axis=1)

## Incluyendo porteros

In [ ]:
# Establecer la columna Name como index porque es una columna identificativa
player_goalkeepers_df = player_df.groupby(['Name'], as_index=True).sum()
player_goalkeepers_df.head()

,7mGoals,7mMissed,6mGoals,6mMissed,WingGoals,WingMissed,BTGoals,BTMissed,FBGoals,FBMissed,...,EGMissed,9mGoals,9mMissed,TF,2Min,AS,ST,BS,Time,scoring
Name,,,,,,,,,,,,,,,,,,,,,
A.HUSAIN Mohamed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,242.066667,-2.25
AAR William,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
ABALO Luc,0.0,0.0,1.0,0.0,11.0,6.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,286.816667,6.90
ABDELFATTAH Omar,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.300000,-1.45
ABDELKADER Rahim,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,3.0,2.0,1.0,2.0,0.0,0.0,141.883333,3.55


## Excluyendo porteros

In [ ]:
player_no_goalkeepers_df = player_df.copy()
player_no_goalkeepers_df = player_no_goalkeepers_df[~player_no_goalkeepers_df['Name'].isin(only_goalkeepers['Name'])]
# Establecer la columna Name como index porque es una columna identificativa
player_no_goalkeepers_df = player_no_goalkeepers_df.groupby(['Name'], as_index=True).sum()
player_no_goalkeepers_df.head()

,7mGoals,7mMissed,6mGoals,6mMissed,WingGoals,WingMissed,BTGoals,BTMissed,FBGoals,FBMissed,...,EGMissed,9mGoals,9mMissed,TF,2Min,AS,ST,BS,Time,scoring
Name,,,,,,,,,,,,,,,,,,,,,
A.HUSAIN Mohamed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,242.066667,-2.25
AAR William,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00
ABALO Luc,0.0,0.0,1.0,0.0,11.0,6.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,1.0,1.0,2.0,0.0,0.0,286.816667,6.90
ABDELFATTAH Omar,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.300000,-1.45
ABDELKADER Rahim,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,3.0,2.0,1.0,2.0,0.0,0.0,141.883333,3.55


# Ejecución del algoritmo